In [ ]:
import pandas as pd
import re

In [ ]:
!mkdir -p data && cd data && wget -c --retry-connrefused --tries=0 --timeout=50 http://aliopentrace.oss-cn-beijing.aliyuncs.com/v2018Traces/batch_task.tar.gz

In [ ]:
!cd data && tar -xvzf batch_task.tar.gz

In [ ]:
# From https://github.com/alibaba/clusterdata/blob/master/cluster-trace-v2018/fetchData.sh
df = pd.read_csv('data/batch_task.csv', names=['task_name', 'instance_num', 'job_name', 'task_type', 'status', 'start_time', 'end_time', 'plan_cpu', 'plan_mem'])
df['duration'] = df['end_time'] - df['start_time']

In [ ]:
# Task name is:
# a) containing dependencies (like 'J4_2_3' -> task 4 depends on 2 and 3)
TASK_NAME_RE = re.compile('^[^_]*[A-Z](?P<task_id>\d+)(_(?P<deps>[\d+_]+))?(_Stg\d+)?$') # Note: sometimes job ends with _Stg*
# b) independent task (like 'task_LTE4NjUxMjg5NDY5MDI4NjAzNzU=')
SINGLE_TASK_RE = re.compile('^task_[a-zA-Z0-9]+=*$')
# c) 'MergeTask'

# check that we cover all cases:
assert df.task_name.apply(lambda f: TASK_NAME_RE.match(f) is not None or SINGLE_TASK_RE.match(f) is not None or f == 'MergeTask').all()

In [ ]:
# Check that 'MergeTask's are in fact independent tasks (i.e. the only task in a job)
def get_merge_task_stats(df):
    df = df[['job_name', 'task_name']].copy()
    df['is_merge_task'] = df['task_name'] == 'MergeTask'
    return df.groupby('job_name').agg(
        count=pd.NamedAgg('task_name', 'count'),
        mergeCount=pd.NamedAgg('is_merge_task', 'sum')
    )

assert len(get_merge_task_stats(df).query('mergeCount > 0 and count > 1')) == 0

In [ ]:
# Extract dependency info
def get_task_index_and_deps(row):
    task_name = row.task_name
    if m := TASK_NAME_RE.match(task_name):
        if m.group('deps'):
            deps = [int(item) for item in m.group('deps').split('_') if item != '']
        else:
            deps = []
        return int(m.group('task_id')), deps
    else:
        return 1, []

df[['task_index', 'task_deps']] = df[['task_name']].apply(get_task_index_and_deps, result_type='expand', axis=1)

In [ ]:
df[['job_name', 'task_index', 'task_deps', 'duration', 'instance_num']].head()

In [ ]:
# Sample job
df[['job_name', 'task_index', 'task_deps', 'duration', 'instance_num']].query('job_name == "j_3"')

In [ ]:
# Filter jobs with 4 or more tasks
jobs = df.groupby("job_name").filter(lambda x: len(x) > 3)

In [ ]:
job_names = jobs["job_name"]

In [ ]:
# Get a sample
sample_job_names = job_names.sample(n=10, random_state=333)
sample_jobs = jobs[jobs["job_name"].isin(sample_job_names)]

In [ ]:
sample_jobs["job_name"].value_counts()


In [ ]:
sample_jobs.query('job_name == "j_1849871"')

In [ ]:
from pathlib import Path
import shutil


MAX_DURATION = 60
INDENT = "    "
DIRECTORY = "data/generated_dags"

def task_template(task_data, job_data) -> (str, [str]):
    dependencies = []
    base_task = f"""
task_{task_data['task_name']} = BashOperator(
    task_id='{task_data['task_name']}',
    bash_command='sleep {min(task_data['duration'], MAX_DURATION)}',
)
    """.strip()
    
    for dependency_index in task_data["task_deps"]:
        dependency = None
        for _, task in job_data.iterrows():
            if task["task_index"] == dependency_index:
                dependency = task
                break
        assert dependency is not None
        
        dependency_template = f"""
{dependency['task_name']} >> {task_data['task_name']}
""".strip()
        dependencies.append(dependency_template)
    
    return base_task, dependencies

for job_name in sample_job_names:
    job_data = sample_jobs[sample_jobs["job_name"] == job_name].copy()
    
    
    imports = f"""
import pendulum
from airflow import DAG
from airflow.operators.bash import BashOperator


with DAG(
    dag_id='{job_name}',
    schedule_interval='*/5 * * * *',
    start_date=pendulum.datetime(2021, 1, 1, tz="UTC"),
    catchup=False,
) as dag:
""".strip()
    
    templated_tasks = []
    templated_dependencies = []
    
    for _, task_data in job_data.iterrows():
        task_templated, dependencies_templated = task_template(task_data, job_data)
        templated_tasks.append(task_templated)
        templated_dependencies.extend(dependencies_templated)
    
    Path(DIRECTORY).mkdir(parents=True, exist_ok=True)
    Path(f'{DIRECTORY}/{job_name}').mkdir(parents=True, exist_ok=True)
    
    with open(f'{DIRECTORY}/{job_name}.py', 'w+') as f:
        f.write(imports)
        
        task_lines = "\n".join(templated_tasks)
        tasks_data = [f"{INDENT}{line}" for line in task_lines.split("\n")]
        tasks = "\n".join(tasks_data)
        
        f.write("\n")
        f.write(tasks)
        
        dependencies_lines = "\n".join(templated_dependencies)
        dependencies_data = [f"{INDENT}{line}" for line in dependencies_lines.split("\n")]
        deps = "\n".join(dependencies_data)
        
        f.write("\n")
        f.write(deps)
    
    shutil.copyfile(f'{DIRECTORY}/{job_name}.py', f'{DIRECTORY}/{job_name}/{job_name}.py')
        
    